<a href="https://colab.research.google.com/github/newb-dev-1008/Blind-Object-Detection/blob/main/Live_Object_Detection_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os 
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

import cv2
cap = cv2.VideoCapture(0)

sys.path.append("..")

In [2]:
!git clone https://github.com/newb-dev-1008/Blind-Object-Detection.git

Cloning into 'Blind-Object-Detection'...
remote: Enumerating objects: 2484, done.
remote: Counting objects: 100% (2484/2484), done.
remote: Compressing objects: 100% (1857/1857), done.
remote: Total 2484 (delta 610), reused 2455 (delta 596), pack-reused 0
Receiving objects: 100% (2484/2484), 32.22 MiB | 6.12 MiB/s, done.
Resolving deltas: 100% (610/610), done.


In [3]:
!pip install tf_slim Cython pandas

     |████████████████████████████████| 358kB 3.8MB/s 


In [4]:
%matplotlib inline
import os
import sys

os.environ['PYTHONPATH'] += ":/content/Blind-Object-Detection/models-master"
sys.path.insert(0, "/content/Blind-Object-Detection/models-master/research")

In [6]:
print(sys.path)

['/content/Blind-Object-Detection/models-master/research', '', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '..']


In [8]:
!python setup.py build
!python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/export_tflite_graph_tf2.py -> build/lib/object_detection
copying object_detection/model_lib_v2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2.py -> build/lib/object_detection
copying object_detection/model_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/model_hparams.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/exporter_lib_v2.py -> build/lib/object_detection
copying object_detection/export_tflite_graph_lib_tf2_test.py -> build/lib/object_detection
copying object_detection/inputs.py -> build/lib/object_detection
copying object_detection/inputs_test.py -> build/lib/object

In [9]:
%cd "/content/Blind-Object-Detection/models-master/research/object_detection/builders"
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

/content/Blind-Object-Detection/models-master/research/object_detection/builders
2021-01-15 11:19:42.586015: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-01-15 11:19:46.343406: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-15 11:19:46.374616: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-15 11:19:46.461439: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-15 11:19:46.461521: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2e3c2708f0ba): /proc/driver/nvidia/version does not exist
2021-01-15 11:19:4

In [10]:
from object_detection.utils import ops as utils_ops
from utils import label_map_util
from utils import visualization_utils as vis_utils 

In [12]:
MODEL_NAME = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
MODEL_FILE = MODEL_NAME  + ".tar.gz"
DOWNLOAD_BASE = "http://download.tensorflow.org/models/object_detection/tf2/20200711/"

PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [17]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for fil in tar_file.getmembers():
  file_name = os.path.basename(fil.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(fil, os.getcwd())

In [18]:
print(file_name)

variables.index


In [19]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.compat.v2.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name = '')

NotFoundError: ignored

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes = 90)
category_index = label_map_util.create_category_index(categories) 